<div style='color: #690027;' markdown="1">
<h1>From leaf to label: stomata detection</h1></div>

This notebook allows you to upload and test your own photomicrographs with the deep learning system discussed in the paper *From leaf to label: a robust automated workflow for stomata detection* by *Sofie Meeus, Jan Van den Bulcke and Francis Wyffels*.<img src="https://drive.google.com/uc?export=view&id=1g1wIt37A07yDi7w9uCza3eMt1oEPLJ5v" alt="Overview" style="width:600px;"/>
As illustrated above, your photomicrograph (A) will be divided into small overlapping patches (B) using a sliding window approach. A deep neural network (VGG19) is trained to label these patches (C). Positively labeled patches of a photomicrograph (D) are clustered which results in the detection (E) depending on the threshold of your choice.
To begin, please run the following cell by clicking the "run" button or by using shift-enter.
Before starting, if you run this notebook on Colab you might want GPU acceleration. Therefore click: *Edit* > *Notebook Settings* > *Hardware accelerator GPU* > *Save*

### Load libraries
We start by loading a few python libraries:
- [PIL](https://pillow.readthedocs.io/en/stable/): a convenient Python imaging library- [numpy](https://numpy.org): het fundamentele pakket voor wetenschappelijk rekenen met Python- [sklearn](https://scikit-learn.org/stable/): the scikit-learn machine learning package, more specifically the clustering functionality- [os](https://docs.python.org/3/library/os.html): a Python library for using operating system dependent functionality, e.g., reading, writing, listing files- [matplotlib](https://matplotlib.org): a Python library for making graphs

In [ ]:
from PIL import Image, ImageEnhance, ImageOpsimport numpy as npfrom sklearn.cluster import MeanShift, estimate_bandwidthimport osimport matplotlib.pyplot as plt

Om te specificeren welke NVidia GPU we zullen gebruiken, moeten we de volgende commando's uitvoeren. Dit gaat ervan uit dat Keras en Tensorflow GPU-enabled zijn. Bekijk de [TensorFlow](https://www.tensorflow.org/install/gpu) documentatie voor meer informatie.

In [ ]:
# If there is a NVidia enabled GPU, run this# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

The code that produced the results presented in the manuscript is based on Tensorflow and Keras. At the time of the writing, TensorFlow 2.0 was not yet available. However, since we relied on a TF 1.0 and Keras implementation. However, the principles for training your own deep learning model stay the same and should be sufficient to get started with [Keras](https://keras.io/getting_started/intro_to_keras_for_researchers/) yourself.

In [ ]:
from tensorflow.keras import backend as Kfrom tensorflow.keras.layers import Input, Convolution2D, Conv2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Densefrom tensorflow.keras.models import Model, load_model# from keras.engine.topology import get_source_inputsfrom tensorflow.keras.utils import get_source_inputsfrom tensorflow.keras.utils import get_filefrom tensorflow.python.keras.utils import layer_utils# import kerasimport tensorflow.keras as kerasfrom tensorflow.keras.preprocessing.image import load_imgfrom tensorflow.keras.preprocessing.image import img_to_arrayfrom tensorflow.keras.preprocessing.image import ImageDataGeneratorimport tensorflow as tf
# limiting GPU VRAMconfig = tf.compat.v1.ConfigProto()config.gpu_options.allow_growth = Truesess = tf.compat.v1.Session(config=config)

### Set data

To train a deep learning model, you need data. As previously discussed, the deep learning model we are discussing here will be trained based on rectangular patches. In order to get a robust model, both positive and negative examples must be presented to the system.
The data needs to be split into three parts:- The training set, i.e., the data which is used for changing the weights of the (deep) neural network;- The validation, i.e., the data which is used to see how well the learning process is going and to tune the model's hyperparameters;- The test data, i.e., the data you feed to the trained system afterwards when the detector is deployed.
This notebook concerns the training and validation of the deep learning system for stomata detection, and a small dataset limited to *Carapa procera* is used for didactic purposes. This also limits the computational needs (a full training with multiple species (cf. the paper) requires more patience.

We start by downloading and unzipping the dataset:

In [ ]:
!wget https://zenodo.org/record/3902280/files/data.zip!unzip 'data.zip'

In [ ]:
train_dir = "./data/training/"val_dir = "./data/validation/"

The training and validation data consists of patches of 120 by 120 pixels. A positive patch shows a stomata:
<img src="https://drive.google.com/uc?export=view&id=11oZG14b8ZnbzooeeQpYaQeUe8wSbErmV"  width="120" />
    
A negative patch of *Carapa procera* has no stomata (or only partially) within the patch:
<img src="https://drive.google.com/uc?export=view&id=1vwcYiZDJffjQ6gXCpThwvbcCoPhgseBS"  width="120" />

In order to obtain these patches you will need labeled microphotographs (i.e., microphotographs of which you have the x,y coordinates of the center position of the stomata that are shown). Based on these labels, the patches can be cropped by using the [crop function of PIL](https://pillow.readthedocs.io/en/stable/reference/Image.html) or simply by [matrix slicing](https://numpy.org/doc/1.18/reference/arrays.indexing.html) in Python Numpy.

The preprocessor defines the data augmentation that will be applied to the dataset. This consists of random rotations, horizontal and vertical flips of the patches:

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=180, horizontal_flip=True, vertical_flip=True, rescale=1/255.)

Apart from the data augmentation that will be applied, we need to set a [ImageDataGenerator](https://keras.io/api/preprocessing/image/#imagedatagenerator-class) that defines the size of the patches (120 x 120 pixels), the color mode, the batch size (i.e., the number of samples used in one training iteration), the classification mode of the task (i.e., binary classification: a patch can be positive or negative), whether the data needs to be shuffled or not and the seed (starting point) of the random number generator. Additionally, you need to provide a path to the directory with the training data.

In [ ]:
batch_size = 128

In [ ]:
train_generator = train_datagen.flow_from_directory(    directory=r"./data/training/",    target_size=(120, 120),    color_mode="rgb",batch_size=batch_size,    class_mode="binary",    shuffle=True,seed=53The assistant didn't receive any text to translate. Your instruction suggests that you were supposed to provide some Dutch text for translation. You might want to try again.

Voor validatie moet je ook een ImageDataGenerator leveren. Dit is gedefinieerd met dezelfde eigenschappen als die voor training, maar zonder data-augmentatie.

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255.)
validation_generator = test_datagen.flow_from_directory(r"./data/validation/",        target_size=(120, 120),        color_mode="rgb",batch_size=batch_size,class_mode='binary')

### Set model parameters

We start from a [VGG19](https://arxiv.org/abs/1409.1556) with two dense layers on top. The convolutional neural layers are pre-trained on [ImageNet](https://ieeexplore.ieee.org/abstract/document/5206848). Consequently, only the dense layers are trained. These pre-trained weights can be downloaded from Keras by adding the keyword "imagenet".

In [ ]:
number_dense_neurons = 2048

In [ ]:
# VGG19 Fine Tunedfrom tensorflow.keras.applications import VGG19# We start from a VGG19 base (convolutional neural layers) with weights pre-trained on ImageNetvgg19_base = VGG19(weights='imagenet',include_top=False,input_shape=(120,120,3))x = vgg19_base.outputx = Flatten()(x)# We add our own densely layered classifier on topx = Dense(2*number_dense_neurons,activation='relu')(x)x = Dropout(0.5)(x)x = Dense(number_dense_neurons,activation='relu')(x)x = Dropout(0.5)(x)# Output layerx = Dense(1,activation='sigmoid')(x)
model = Model(inputs=vgg19_base.input, outputs=x)
# Only the dense layers are trained, hence, the convolutional neural layers are set not trainable:for layer in vgg19_base.layers:    layer.trainable = False
# What does the network look likemodel.summary()

### Train model

These parameters were optimized by using the [Adam](https://arxiv.org/pdf/1412.6980.pdf) learning rule for which the learning rate was tuned and finally set to 0.000005. Additionally, we need to configure the training loss and metrics for validation.

In [ ]:
learning_rate = 0.000005# Initiate Stochastic Gradient Descent with momentum, learning rate to tuneopt = tf.keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)# Define the losses and metrics for validationmodel.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])

Training can be done by using the function *fit*. We train for 50 epochs. Note that we configured our architecture in a way that only the weights of the dense layers are adjusted.

In [ ]:
epochs = 15The input provided does not contain any Dutch text. It appears to be a part of a Python code which does not need translation.train_generator,        epochs=epochs,        validation_data=validation_generator)

Nu zijn we klaar, ons netwerk is getraind en kan in gebruik worden genomen. Om je systeem te kunnen gebruiken, moet je de modelparameters opslaan. Dit kan door *model.save(pad)* aan te roepen met *pad*, het pad naar het bestand waarin je de parameters wilt opslaan. Bovendien geeft de *fit* een *history* object terug. Dit registreert de trainings- en validatievoortgang over de epochs. Dit is dan ook handig om de trainingsvoortgang bij te houden, bijvoorbeeld bij het vergelijken van meerdere hyperparameterinstellingen.

In [ ]:
# Save the Carapa procera deep learning modelmodel.save('my_carapa_procera_model')
# Plot the training and validation losses over timeplt.plot(history.history['loss'], label='Training loss')plt.plot(history.history['val_loss'], label='Validation loss')

### Load a deep learning model
Now we have trained a first deep learning model on the *Carapa procera*. This is stored in the object *model*. If you want to start from a saved deep learning model, you simply load it from a file by calling: *model = load_model(path_to_model)*.

### Image and detection parameters
Het model maakt gebruik van een schuifvensterbenadering. Hoewel niet het meest (computationeel) efficiënt, is het heel eenvoudig te begrijpen. Het venster heeft een grootte van 120 bij 120 pixels en we gebruiken een stap van 10 pixels. We beginnen met het laden van onze afbeelding:

In [ ]:
demo_image = './data/Carapa_procero_demo.jpg' # you can use any other Carapa procero microphotograph

In [ ]:
image = Image.open(demo_image)fig, ax = plt.subplots(figsize=(20, 10))image = np.array(image) # conversion to a Numpy arrayax.imshow(image)

In [ ]:
shift = 10patch_size = 120

Also part of the detection parameters is the number of slides we will do:

In [ ]:
no_x_shifts = (np.shape(image)[0] - patch_size) // shiftno_y_shifts = (np.shape(image)[1] - patch_size) // shiftprint("We will do "+str(no_x_shifts*no_y_shifts)+" slides. Consequently, the deep learning model will be applied to "+str(no_x_shifts*no_y_shifts)+" windows.")

### Classification with a deep learning model
Now that we have identified all the windows, we apply the deep learning model. This is done by calling the *predict* function. The image we apply needs to be converted and normalized before applying the deep learning model. Since the output of the deep learning model is between 0 and 1, we also have to set a threshold from which we accept the output as a positive classification. The higher this threshold, the more precise the system will be in detecting stomata. However, if the threshold is too high, the system won't be able to detect stomata at all. Here we use the same threshold as determined in the paper:

In [ ]:
threshold = 0.7

In [ ]:
"""
stomata = []offset = patch_size // 2for x in np.arange(no_x_shifts + 1):    for y in np.arange(no_y_shifts + 1):# center of the windowx_c = x * shift + offset        y_c = y * shift + offset        
# extraction of the window and conversion before applying the deep learning modelpatch = image[x_c - offset:x_c + offset, y_c - offset:y_c + offset, :]        patch = patch.astype('float32')patch /= 255        
# applying the deep learning modely_model = model.predict(np.expand_dims(patch, axis=0))
# if the output of the network is above the        if y_model[0] > threshold:stomata.append([x_c, y_c])"""

In [ ]:
patches = []coordinates = []stomata = []offset = patch_size // 2for x in np.arange(no_x_shifts + 1):for y in np.arange(no_y_shifts + 1):# Center of the window        x_c = x * shift + offsety_c = y * shift + offset
# Cutting out the window and converting to expected format before applying the deep learning modelpatch = image[x_c - offset:x_c + offset, y_c - offset:y_c + offset, :]patch = patch.astype("float32")patch /= 255        
# Add window to the list of windows        patches.append(np.expand_dims(patch, axis=0))        
# Add coordinates of the center of the window to the list of coordinates        coordinates.append([x_c, y_c])
# Process all windows in batches of 128 windows to carry out calculations in parallelbatch_size = 128for b in range(0, len(patches), batch_size):    batch = patches[b:b + batch_size]
# Apply the model to perform the detectiony_model = model.predict(np.vstack(batch))
for p in range(0, len(y_model)):# Stoma if the output of the model lies above the threshold        if y_model[p] > threshold:stomata.append(coordinates[b + p])

### Clustering the detected stomata
All positively labeled patches are clustered using mean shift clustering. This technique groups neighboring (or even overlapping) positively labeled patches from which the resulting stoma coordinates are derived. Therefore, we can rely on the package [MeanShift](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html) which is available in [scikit-learn](https://scikit-learn.org).

In [ ]:
bandwidth = patch_size // 2
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)ms.fit(stomata)stomata = np.array([[x[1], x[0]] for x in ms.cluster_centers_]) # cluster_centers_ is inverted

### Plotting the results

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))ax.imshow(image)ax.plot(stomata[:,0], stomata[:,1], 'xr', alpha=0.75, markeredgewidth=3, markersize=12)